In [1]:
import os, sys
import numpy as np
import time
import pickle
import pandas as pd
import os
from deploy_utils import HiddenPrints, run_deployment, extend_material_parameters, get_paths
# os.environ["TEXMF"] = r"C:\Users\vbalmer\AppData\Local\Programs\MiKTeX"

-------------------------------------------------------
Start Executing FEM_Q Script
-------------------------------------------------------


Seed set to 42


In [2]:
numpy_sampler = False
single_sample = True

# define location of trained model and input data to be used
# path_collection = get_paths(vnum='182', epnum='19926')                                                # for lin.el. RC case
# path_collection = get_paths(vnum='198', epnum='9997')                                                 # for glass case   
path_collection = get_paths(vnum='283', epnum='49879', vnumD='283', epnumD='49879')                     # for nonlin. RC case  


In [3]:
# Inputs for iteration

# If the data comes from the sampler:
if numpy_sampler:
    path = '..\\01_SamplingFeatures'
    # name = 'data_240724_1752_case4\outfile.npy'
    name = 'data_20241104_1545_case8\outfile.npy'
    features = np.load(os.path.join(path, name))
    mat_tot_dict_ = {
        'L': features[:,0],         # length
        'B': features[:,1],         # width
        'E_1': features[:,2],       # Young's modulus steel / glass / concrete
        'E_2': features[:,3],       # Young's modulus - / interlayer / reinforcing steel
        'ms': features[:,4],        # mesh_size
        'F': features[:,5],         # force_magnitude
        's': features[:,6],         # scenario 0...6
        't_1': features[:,7],       # thickness of the plate
        't_2': features[:,8],       # thickness of plate
        'nl': features[:,9],        # amount of layers
        'nu_1': features[:,10],     # Poisson's ratio
        'nu_2': features[:,11],     # Poisson's ratio
        'mat': features[:,12]       # Material type     (1 = lin.el., 3 = CMM, 10 = glass)
    }

elif numpy_sampler == False and not single_sample: 
    path = '..\\01_SamplingFeatures'
    name = 'output\\data_20241111_0904_case10\\outfile.pkl'
    with open(os.path.join(path, name),'rb') as handle:
            in_dict = pickle.load(handle)
    mat_tot_dict_ = in_dict

if single_sample:
    # # pure shear force (for RC linear, mat = 1, scenario 8)
    # mat_tot_dict_ = {
    #     'L': np.array([7500]),
    #     'B': np.array([7500]),
    #     'E_1': np.array([33600]),
    #     'E_2': np.array([0]),
    #     'ms': np.array([750]),
    #     'F': np.array([750000]),       #=10*L
    #     'F_N': np.array([0]),         # not required in this case
    #     's': np.array([8]),
    #     't_1': np.array([200]),
    #     't_2': np.array([0]),
    #     'nl': np.array([20]),
    #     'nu_1': np.array([0.2]),
    #     'nu_2': np.array([0]),
    #     'mat': np.array([1])
    # }

    # # moment + normal force (for RC linear, mat = 1, scenario 11)
    # mat_tot_dict_ = {
    #     'L': np.array([7500]),
    #     'B': np.array([7500]),
    #     'E_1': np.array([33600]),
    #     'E_2': np.array([0]),
    #     'ms': np.array([750]),
    #     'F': np.array([0.015]),           # Uniformly distributed load in z-direction
    #     'F_N': np.array([750000]),       # Normal force (n[N/m]*L); e.g. n=50kN/m --> F_N = 100*7500 = 750000
    #     's': np.array([11]),
    #     't_1': np.array([200]),
    #     't_2': np.array([0]),
    #     'nl': np.array([20]),
    #     'nu_1': np.array([0.2]),
    #     'nu_2': np.array([0]),
    #     'mat': np.array([1])
    # }

    # bending 2D (for glass, mat = 10, scenario 20)
    # mat_tot_dict_ = {
    #     'L': np.array([1500]),
    #     'B': np.array([1500]),
    #     'E_1': np.array([70000]),
    #     'E_2': np.array([300]),
    #     'ms': np.array([150]),
    #     'F': np.array([0.005]),
    #     'F_N': np.array([0]),       # not required in this case.
    #     's': np.array([20]),
    #     't_1': np.array([5]),
    #     't_2': np.array([0.4]),
    #     'nl': np.array([5]),
    #     'nu_1': np.array([0.23]),
    #     'nu_2': np.array([0.5]),
    #     'mat': np.array([10])
    # }

    # bending 1D (for RC nonlinear, mat = 3, scenario 10)
    mat_tot_dict_ = {
        'L': np.array([7500]),
        'B': np.array([7500]),
        'CC': np.array([1]),
        'E_1': np.array([0]),
        'E_2': np.array([0]),
        'ms': np.array([750]),
        'F': np.array([0.025]),
        'F_N': np.array([0]),       # not required in this case.
        's': np.array([10]),
        't_1': np.array([200]),
        't_2': np.array([0]),
        'nl': np.array([20]),
        'nu_1': np.array([0]),
        'nu_2': np.array([0]),
        'mat': np.array([3]),
        'rho': np.array([0.04])
    }



if mat_tot_dict_['mat'] == 3:
    mat_tot_dict = extend_material_parameters(mat_tot_dict_)
else: 
    mat_tot_dict = mat_tot_dict_

mat_tot_raw = pd.DataFrame.from_dict(mat_tot_dict)

if not single_sample:
    # import mat_res.pkl from data that was used for training of algorithm (if it comes from global sampling..)
    path_mat_res = '..\\02_Simulator'
    # name = 'Simulator\\results\saved_runs\data_240805_1134_case4\mat_res.pkl'        # take 240805 here, as there the SN are included
    # name = 'Simulator\\results\saved_runs\data_20241104_1854_case8\mat_res.pkl'
    name = 'Simulator\\results\saved_runs\data_20241111_1101_case10\mat_res.pkl'
    with open(os.path.join(path_mat_res, name),'rb') as handle:
        mat_res = pickle.load(handle)

    # Choose the simulation number(s) that shall be tested
    desired_SN = [21, 32, 45, 74, 76, 82]
    mat_tot = mat_tot_raw.iloc[[np.where(mat_res['SN'] == i)[0][0] for i in desired_SN]]
    mat_tot.reset_index(drop=True, inplace=True)
    print('As a check, in mat_tot (from sampling) (t1_tot = ', round(mat_tot['t_1'][0], 1), 'mm) should be equal to t1 in mat_res (directly read) (t1_res = ', round(mat_res['t_1'][desired_SN[0]],1), 'mm).')
else: 
    mat_tot = mat_tot_raw

print(mat_tot)

      L     B  CC      E_1  E_2   ms      F  F_N   s  t_1  ...  fsu        Es  \
0  7500  7500   1  33600.0    0  750  0.025    0  10  200  ...  470  205000.0   

      Esh   D  tb0  tb1      ect     ec0  fcp  fct  
0  8000.0  16  5.8  2.9  0.00009  0.0023   30  2.9  

[1 rows x 27 columns]


In [4]:
###########################################
# Deployment with NN
###########################################

conv_plt = True
simple = True                          # Leave this on "True" for deployment.
samples = int(mat_tot.shape[0])
n_simple = 1                           # can also be len(desired_SN) if imported data from simulations (not single_sample)
NN_hybrid = {
    'predict_D': True,                # If true, solves the system with NN_hybrid solver (prediction of D); if False: "normal" solver
    'predict_sig': True,                # If true, solves the system with NN_hybrid solver (prediction of sig); if False: "normal" solver
    }

####
# Note: predict_D should not be used in lin.el. case, as there is only one initialisation and this happens with lin.el. model. 
# => for lin.el. always set predict_D = False (glass / steel / RC)
# => for nonlin: can choose what should be predicted.
####

mat_res = run_deployment(mat_tot, conv_plt, simple, n_simple, NN_hybrid, path_collection)
mat_res_pd_NN = pd.DataFrame.from_dict(mat_res)
mat_res_pd_NN[['ms', 'L', 'B']].head()

-------------------------------------------------------
1 Start Meshing with gmsh
-------------------------------------------------------
1.1 Assemble Model and mesh
1.2 Boundary Conditions and Loads
1.3 Read Material, Constitutive Model and Integration Options
1.4 Postprocess Mesh
   1.41 Node Coordinates
   1.42 Node Connectivity
   1.43 Element Center Information 1/2
   1.44 Element Numbering
   1.45 Global Coordinate Transformation
   1.46 Node Information
   1.47 Flip Element Arrangement Clockwise
   1.48 Element Center Information 2/2
   1.49 Element Connectivity
   1.410 Element Integration Point Information
   1.411 Geometrical Information per Element
   1.412 Material Information per Element
   1.413 Masks for Nodes in Areas
   1.414 Coplanar Nodes
-------------------------------------------------------
2 Initialisation
-------------------------------------------------------
2.1 Assembly of force vector and condensed DOFs
2.2 Solution for Linear Elasticity
Gauss order for NN-r

IndexError: index 2 is out of bounds for axis 3 with size 2

In [ ]:
###########################################
# Deployment without NN (= ground truth)
###########################################

conv_plt = True
simple = True  
samples = int(mat_tot.shape[0])
n_simple = 1
NN_hybrid_2 = {'predict_D': False,
             'predict_sig': False}

mat_res = run_deployment(mat_tot, conv_plt, simple, n_simple, NN_hybrid_2, path_collection)
mat_res_pd = pd.DataFrame.from_dict(mat_res)
mat_res_pd[['ms', 'L', 'B']].head()

In [ ]:
from main_utils_vb import all_eps_sig_plots, imshow_plots, all_De_plots

eh_cum_NN = mat_res_pd_NN[['eh_cum']].to_numpy()[0,0]
eh_cum_NN_ = mat_res_pd_NN[['eh_cum_']].to_numpy()[0,0]
eh_cum_NLFEA = mat_res_pd[['eh_cum']].to_numpy()[0,0]

if NN_hybrid['predict_sig'] and not NN_hybrid['predict_D']:
    sh_cum_NN = mat_res_pd_NN[['sh_cum']].to_numpy()[0,0]
    sh_cum_NN_ = mat_res_pd_NN[['sh_cum_']].to_numpy()[0,0]    
    sh_cum_NLFEA = mat_res_pd[['sh_cum']].to_numpy()[0,0]
    coord = mat_res_pd_NN[['COORD']]

    # adjust plotting function that if NLFEA vs NN --> plots all three things in one.
    all_eps_sig_plots(7, 7, eh_cum_NN, 0, sh_cum_NN, eh_cum_NN_, sh_cum_NN_,eh_cum_NLFEA,sh_cum_NLFEA, NN = 'NN', tag = 51)

if not NN_hybrid['predict_sig'] and NN_hybrid['predict_D']:
    De_cum_NN = mat_res_pd_NN[['De_cum']].to_numpy()[0,0]
    De_cum_NN_ = mat_res_pd_NN[['De_cum_']].to_numpy()[0,0]
    De_cum_NLFEA = mat_res_pd[['De_cum']].to_numpy()[0,0]

    all_De_plots(6, 6, eh_cum_NN, eh_cum_NN_, eh_cum_NLFEA, De_cum_NN, De_cum_NN_, De_cum_NLFEA, tag = 'max')


In [ ]:
# if data should be saved to folder instead of being overwritten with the next simulation, use save_folder = True

from datetime import datetime
import shutil
from data_work_depl import create_vid
import os

save_folder = True

if save_folder:
    if NN_hybrid['predict_sig'] and NN_hybrid['predict_D']:
        relative_path = ['data_out\\mat_res_norm.pkl', 'data_out\\mat_res_NN.pkl']
    elif NN_hybrid['predict_sig']:
        relative_path = ['data_out\\mat_res_norm.pkl', 'data_out\\mat_res_NN_sig.pkl']
    elif NN_hybrid['predict_D']:
        relative_path = ['data_out\\mat_res_norm.pkl', 'data_out\\mat_res_NN_D.pkl']
    
    folder_paths = ['plots\\mike_plots', 'plots\\diagonal_plots', 'plots\\conv_plots', 'plots\\eps_sig_plots_it', 'plots\\De_plots_it']

    current_time = datetime.now()
    new_folder = current_time.strftime("data_%Y%m%d_%H%M_case" + 'xx')
    new_folder_path = os.path.join('data_out', new_folder)
    os.makedirs(new_folder_path, exist_ok=True)


    # Copy files
    for i, file_path in enumerate(relative_path):
        destination_path = os.path.join(new_folder_path, os.path.basename(file_path))
        shutil.copy(file_path, destination_path)
        print(f'File {i + 1} copied to {destination_path}')

    # Copy folders
    for folder in folder_paths:
        source_folder = os.path.abspath(folder)
        destination_folder = os.path.join(new_folder_path, os.path.basename(folder))
        shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)
        print(f'Folder "{folder}" copied to {destination_folder}')
    
    create_vid(os.path.abspath('plots\\diagonal_plots'), os.path.join(new_folder_path, 'diag_plots_video.mp4'), fps = 1)

In [ ]:
# print(mat_res_pd.keys())
# print(mat_res_pd['eps_g'][0].shape)
# print(mat_res_pd['eps_g'][0][:,0,0,0]) # these should correspond to eps_x
# print(mat_res_pd['eps_g'][0][:,0,0,7]) # these should correspond to gamma_xz
# print(mat_res_pd['ms'])
# print(mat_res_pd['t_1'])